### 신경망 파라미터 튜닝하기

In [16]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train,y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [26]:
X_new = X_test[:3]

In [27]:
def build_model(n_hidden=1 ,n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse',optimizer=optimizer)
    return model

In [28]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [29]:
keras_reg.fit(X_train, y_train, epochs=100, 
             validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100
363/363 [==============================] - 0s 1ms/step - loss: 1.2015 - val_loss: 0.7104
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6664 - val_loss: 0.6153
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5989 - val_loss: 0.5618
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5399 - val_loss: 0.5180
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5048 - val_loss: 0.4847
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4854 - val_loss: 0.4890
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4831 - val_loss: 0.4570
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4542 - val_loss: 0.4469
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4457 - val_loss: 0.4389
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4418 - val_lo

363/363 [==============================] - 0s 1ms/step - loss: 0.3357 - val_loss: 0.3406
Epoch 82/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3347 - val_loss: 0.3420
Epoch 83/100
363/363 [==============================] - 0s 998us/step - loss: 0.3338 - val_loss: 0.3386
Epoch 84/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3446 - val_loss: 0.3403
Epoch 85/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3354 - val_loss: 0.3386
Epoch 86/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3326 - val_loss: 0.3367
Epoch 87/100
363/363 [==============================] - 0s 994us/step - loss: 0.3359 - val_loss: 0.3371
Epoch 88/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3317 - val_loss: 0.3528
Epoch 89/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3327 - val_loss: 0.3367
Epoch 90/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3326 - val_lo

### RandomizedSearchCV
- 하이퍼파라미터가 많기 때문에 그리드 탐색보다 랜덤 탐색을 사용하는 것이 좋습니다. 
- k fold 교차 검증을 실시함

In [30]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [32]:
param_distribs = {
    "n_hidden":[0,1,2,3,],
    'n_neurons':np.arange(1, 100),
    'learning_rate':reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=2, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                 validation_data=(X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
242/242 [==============================] - 0s 1ms/step - loss: 2.8516 - val_loss: 1.9571
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 1.4973 - val_loss: 1.3301
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 1.1405 - val_loss: 1.0930
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.9809 - val_loss: 0.9592
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.8881 - val_loss: 0.8762
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.8274 - val_loss: 0.8200
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7821 - val_loss: 0.7775
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7472 - val_loss: 0.7435
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7192 - val_loss: 0.7155
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6955 - val_lo

242/242 [==============================] - 0s 1ms/step - loss: 0.4093 - val_loss: 0.4181
Epoch 81/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4083 - val_loss: 0.4170
Epoch 82/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4072 - val_loss: 0.4164
Epoch 83/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4062 - val_loss: 0.4146
Epoch 84/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4051 - val_loss: 0.4134
Epoch 85/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4042 - val_loss: 0.4127
Epoch 86/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4031 - val_loss: 0.4116
Epoch 87/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4021 - val_loss: 0.4107
Epoch 88/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4010 - val_loss: 0.4096
Epoch 89/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4002 - val_loss: 

242/242 [==============================] - 0s 1ms/step - loss: 0.4237 - val_loss: 0.4212
Epoch 60/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4222 - val_loss: 0.4209
Epoch 61/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4209 - val_loss: 0.4184
Epoch 62/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4195 - val_loss: 0.4200
Epoch 63/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4183 - val_loss: 0.4160
Epoch 64/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4171 - val_loss: 0.4155
Epoch 65/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4159 - val_loss: 0.4145
Epoch 66/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4148 - val_loss: 0.4139
Epoch 67/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4136 - val_loss: 0.4125
Epoch 68/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4126 - val_loss: 

242/242 [==============================] - 0s 1ms/step - loss: 0.4886 - val_loss: 0.4869
Epoch 38/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4856 - val_loss: 0.4845
Epoch 39/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4829 - val_loss: 0.4817
Epoch 40/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4802 - val_loss: 0.4790
Epoch 41/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4776 - val_loss: 0.4762
Epoch 42/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4752 - val_loss: 0.4740
Epoch 43/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4727 - val_loss: 0.4726
Epoch 44/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4705 - val_loss: 0.4695
Epoch 45/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4680 - val_loss: 0.4674
Epoch 46/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4658 - val_loss: 

242/242 [==============================] - 0s 1ms/step - loss: 0.5186 - val_loss: 0.5187
Epoch 17/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5163 - val_loss: 0.5140
Epoch 18/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5228 - val_loss: 0.5251
Epoch 19/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5137 - val_loss: 0.5151
Epoch 20/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5278 - val_loss: 0.5325
Epoch 21/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5151 - val_loss: 0.5105
Epoch 22/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5273 - val_loss: 0.5267
Epoch 23/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5195 - val_loss: 0.5198
Epoch 24/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5139 - val_loss: 0.5109
Epoch 25/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5257 - val_loss: 

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000002B87FC81488>, as the constructor either does not set or modifies parameter learning_rate

In [33]:
rnd_search_cv.best_params_

{'learning_rate': 0.00047111403973037276, 'n_hidden': 2, 'n_neurons': 32}

In [34]:
rnd_search_cv.best_score_

-0.3956443965435028